nir_annotation_files_split_0 = [
    "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_0.test.csv",
    "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_0.train.csv",
    "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_0.val.csv"
]

nir_annotation_files_split_1 = [
    "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_1.test.csv",
    "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_1.train.csv",
    "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_1.val.csv"
]

nir_annotation_files_split_2 = [
    "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_2.test.csv",
    "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_2.train.csv",
    "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_2.val.csv"
]

# For Split_0

In [1]:
import os
import cv2
import csv
import time
import concurrent.futures

class VideoFrameExtractor:
    """
    A class to extract frames from video files based on annotations and process these annotations in parallel.
    
    Attributes:
        data_dir (str): The directory where the video files are stored.
        root_dataset_dir (str): The root directory where the extracted frames will be saved.
        annotation_files (dict): A dictionary mapping dataset splits to their corresponding annotation file paths.
    """
    
    def __init__(self, data_dir, root_dataset_dir, annotation_files):
        """
        Initializes the VideoFrameExtractor with dataset directories and annotation file paths.
        """
        self.data_dir = data_dir
        self.root_dataset_dir = root_dataset_dir
        self.annotation_files = annotation_files

    def extract_frames(self, row, output_dir, max_frames_per_chunk):
        """
        Extracts frames from a video file based on a row from the annotation file.
        
        Args:
            row (list): A list containing annotation information for a video segment.
            output_dir (str): The directory where the extracted frames should be saved.
            max_frames_per_chunk (int): The maximum number of frames to extract per chunk.
        """
        try:
            participant_id, file_id, annotation_id, frame_start, frame_end, activity, chunk_id = row
            video_filepath = os.path.join(self.data_dir, file_id + '.mp4')
            new_file_id = file_id.replace("/", "_")
            updated_output_dir = os.path.join(output_dir, f'{activity}', f'{participant_id}_{new_file_id}_frames_{frame_start}_{frame_end}_ann_{annotation_id}_chunk_{chunk_id}')
            
            os.makedirs(updated_output_dir, exist_ok=True)
            cap = cv2.VideoCapture(video_filepath)
            frame_count = 0
             
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_start))
            
            for frame_num in range(int(frame_start), int(frame_end) + 1):
                ret, frame = cap.read()
                if not ret:
                    print(f"Frame number {frame_num} is missing.")
                    break
                output_filename = f'img_{frame_num:06d}.png'
                output_path = os.path.join(updated_output_dir, output_filename)
                cv2.imwrite(output_path, frame)
                frame_count += 1
                
                if frame_count > max_frames_per_chunk:
                    break

            cap.release()
        except Exception as e:
            print(f"Error processing file {file_id}: {e}")

    def process_annotations(self, dataset_sub_dir, max_frames_per_chunk):
        """
        Processes the annotation file for a dataset split in parallel using multiple processes.
        
        Args:
            dataset_sub_dir (str): The sub-directory (e.g., 'train', 'test') for saving extracted frames.
            max_frames_per_chunk (int): The maximum number of frames to extract per chunk.
        """
        annotation_file = self.annotation_files[dataset_sub_dir]
        output_dir = os.path.join(self.root_dataset_dir, dataset_sub_dir)
        
        try:
            with open(annotation_file, 'r') as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  # Skip header row
                
                with concurrent.futures.ProcessPoolExecutor() as executor:
                    futures = [executor.submit(self.extract_frames, row, output_dir, max_frames_per_chunk) for row in reader]
                    concurrent.futures.wait(futures)
        except Exception as e:
            print(f"Error processing annotation file {annotation_file}: {e}")

def format_time(seconds):
    """Converts time in seconds to hours, minutes, and seconds format."""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds"

def main():
    # path to the video files
    data_dir = "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/inner_mirror"
    # the output directory where the extracted frames will be stored 
    root_dataset_dir = "/net/polaris/storage/deeplearning/sur_data/nir_front_top_daa/split_0"
    annotation_files = {
        'train': "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_0.train.csv",
        'test': "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_0.test.csv",
        'val': "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_0.val.csv"
    }

    video_frame_extractor = VideoFrameExtractor(data_dir, root_dataset_dir, annotation_files)
    start_time = time.time()

    for dataset_sub_dir in annotation_files.keys():
        # Chunk = 3 seconds video, 1 second = Frame Rate, Frames: seconds x Frame rate
        max_frames_per_chunk = 95  # This value can be adjusted as needed according to 15 FPS or 30FPS
        print(f"Processing {dataset_sub_dir} dataset...")
        video_frame_extractor.process_annotations(dataset_sub_dir, max_frames_per_chunk)

    end_time = time.time()
    time_taken = format_time(end_time - start_time)
    print(f"Time taken for extracting dataset frames: {time_taken}")

if __name__ == "__main__":
    main()


Processing train dataset...
Processing test dataset...
Processing val dataset...
Time taken for extracting dataset frames: 1 hours, 28 minutes, 55 seconds


# Split_1

In [2]:
import os
import cv2
import csv
import time
import concurrent.futures

class VideoFrameExtractor:
    """
    A class to extract frames from video files based on annotations and process these annotations in parallel.
    
    Attributes:
        data_dir (str): The directory where the video files are stored.
        root_dataset_dir (str): The root directory where the extracted frames will be saved.
        annotation_files (dict): A dictionary mapping dataset splits to their corresponding annotation file paths.
    """
    
    def __init__(self, data_dir, root_dataset_dir, annotation_files):
        """
        Initializes the VideoFrameExtractor with dataset directories and annotation file paths.
        """
        self.data_dir = data_dir
        self.root_dataset_dir = root_dataset_dir
        self.annotation_files = annotation_files

    def extract_frames(self, row, output_dir, max_frames_per_chunk):
        """
        Extracts frames from a video file based on a row from the annotation file.
        
        Args:
            row (list): A list containing annotation information for a video segment.
            output_dir (str): The directory where the extracted frames should be saved.
            max_frames_per_chunk (int): The maximum number of frames to extract per chunk.
        """
        try:
            participant_id, file_id, annotation_id, frame_start, frame_end, activity, chunk_id = row
            video_filepath = os.path.join(self.data_dir, file_id + '.mp4')
            new_file_id = file_id.replace("/", "_")
            updated_output_dir = os.path.join(output_dir, f'{activity}', f'{participant_id}_{new_file_id}_frames_{frame_start}_{frame_end}_ann_{annotation_id}_chunk_{chunk_id}')
            
            os.makedirs(updated_output_dir, exist_ok=True)
            cap = cv2.VideoCapture(video_filepath)
            frame_count = 0
             
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_start))
            
            for frame_num in range(int(frame_start), int(frame_end) + 1):
                ret, frame = cap.read()
                if not ret:
                    print(f"Frame number {frame_num} is missing.")
                    break
                output_filename = f'img_{frame_num:06d}.png'
                output_path = os.path.join(updated_output_dir, output_filename)
                cv2.imwrite(output_path, frame)
                frame_count += 1
                
                if frame_count > max_frames_per_chunk:
                    break

            cap.release()
        except Exception as e:
            print(f"Error processing file {file_id}: {e}")

    def process_annotations(self, dataset_sub_dir, max_frames_per_chunk):
        """
        Processes the annotation file for a dataset split in parallel using multiple processes.
        
        Args:
            dataset_sub_dir (str): The sub-directory (e.g., 'train', 'test') for saving extracted frames.
            max_frames_per_chunk (int): The maximum number of frames to extract per chunk.
        """
        annotation_file = self.annotation_files[dataset_sub_dir]
        output_dir = os.path.join(self.root_dataset_dir, dataset_sub_dir)
        
        try:
            with open(annotation_file, 'r') as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  # Skip header row
                
                with concurrent.futures.ProcessPoolExecutor() as executor:
                    futures = [executor.submit(self.extract_frames, row, output_dir, max_frames_per_chunk) for row in reader]
                    concurrent.futures.wait(futures)
        except Exception as e:
            print(f"Error processing annotation file {annotation_file}: {e}")

def format_time(seconds):
    """Converts time in seconds to hours, minutes, and seconds format."""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds"

def main():
    # path to the video files
    data_dir = "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/inner_mirror"
    # the output directory where the extracted frames will be stored 
    root_dataset_dir = "/net/polaris/storage/deeplearning/sur_data/nir_front_top_daa/split_1"
    annotation_files = {
        'train': "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_1.train.csv",
        'test': "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_1.test.csv",
        'val': "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_1.val.csv"
    }

    video_frame_extractor = VideoFrameExtractor(data_dir, root_dataset_dir, annotation_files)
    start_time = time.time()

    for dataset_sub_dir in annotation_files.keys():
        # Chunk = 3 seconds video, 1 second = Frame Rate, Frames: seconds x Frame rate
        max_frames_per_chunk = 95  # This value can be adjusted as needed according to 15 FPS or 30FPS
        print(f"Processing {dataset_sub_dir} dataset...")
        video_frame_extractor.process_annotations(dataset_sub_dir, max_frames_per_chunk)

    end_time = time.time()
    time_taken = format_time(end_time - start_time)
    print(f"Time taken for extracting dataset frames: {time_taken}")

if __name__ == "__main__":
    main()


Processing train dataset...
Processing test dataset...
Processing val dataset...
Time taken for extracting dataset frames: 1 hours, 28 minutes, 33 seconds


# Split_2

In [3]:
import os
import cv2
import csv
import time
import concurrent.futures

class VideoFrameExtractor:
    """
    A class to extract frames from video files based on annotations and process these annotations in parallel.
    
    Attributes:
        data_dir (str): The directory where the video files are stored.
        root_dataset_dir (str): The root directory where the extracted frames will be saved.
        annotation_files (dict): A dictionary mapping dataset splits to their corresponding annotation file paths.
    """
    
    def __init__(self, data_dir, root_dataset_dir, annotation_files):
        """
        Initializes the VideoFrameExtractor with dataset directories and annotation file paths.
        """
        self.data_dir = data_dir
        self.root_dataset_dir = root_dataset_dir
        self.annotation_files = annotation_files

    def extract_frames(self, row, output_dir, max_frames_per_chunk):
        """
        Extracts frames from a video file based on a row from the annotation file.
        
        Args:
            row (list): A list containing annotation information for a video segment.
            output_dir (str): The directory where the extracted frames should be saved.
            max_frames_per_chunk (int): The maximum number of frames to extract per chunk.
        """
        try:
            participant_id, file_id, annotation_id, frame_start, frame_end, activity, chunk_id = row
            video_filepath = os.path.join(self.data_dir, file_id + '.mp4')
            new_file_id = file_id.replace("/", "_")
            updated_output_dir = os.path.join(output_dir, f'{activity}', f'{participant_id}_{new_file_id}_frames_{frame_start}_{frame_end}_ann_{annotation_id}_chunk_{chunk_id}')
            
            os.makedirs(updated_output_dir, exist_ok=True)
            cap = cv2.VideoCapture(video_filepath)
            frame_count = 0
             
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_start))
            
            for frame_num in range(int(frame_start), int(frame_end) + 1):
                ret, frame = cap.read()
                if not ret:
                    print(f"Frame number {frame_num} is missing.")
                    break
                output_filename = f'img_{frame_num:06d}.png'
                output_path = os.path.join(updated_output_dir, output_filename)
                cv2.imwrite(output_path, frame)
                frame_count += 1
                
                if frame_count > max_frames_per_chunk:
                    break

            cap.release()
        except Exception as e:
            print(f"Error processing file {file_id}: {e}")

    def process_annotations(self, dataset_sub_dir, max_frames_per_chunk):
        """
        Processes the annotation file for a dataset split in parallel using multiple processes.
        
        Args:
            dataset_sub_dir (str): The sub-directory (e.g., 'train', 'test') for saving extracted frames.
            max_frames_per_chunk (int): The maximum number of frames to extract per chunk.
        """
        annotation_file = self.annotation_files[dataset_sub_dir]
        output_dir = os.path.join(self.root_dataset_dir, dataset_sub_dir)
        
        try:
            with open(annotation_file, 'r') as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  # Skip header row
                
                with concurrent.futures.ProcessPoolExecutor() as executor:
                    futures = [executor.submit(self.extract_frames, row, output_dir, max_frames_per_chunk) for row in reader]
                    concurrent.futures.wait(futures)
        except Exception as e:
            print(f"Error processing annotation file {annotation_file}: {e}")

def format_time(seconds):
    """Converts time in seconds to hours, minutes, and seconds format."""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds"

def main():
    # path to the video files
    data_dir = "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/inner_mirror"
    # the output directory where the extracted frames will be stored 
    root_dataset_dir = "/net/polaris/storage/deeplearning/sur_data/nir_front_top_daa/split_2"
    annotation_files = {
        'train': "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_2.train.csv",
        'test': "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_2.test.csv",
        'val': "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/inner_mirror/midlevel.chunks_90.split_2.val.csv"
    }

    video_frame_extractor = VideoFrameExtractor(data_dir, root_dataset_dir, annotation_files)
    start_time = time.time()

    for dataset_sub_dir in annotation_files.keys():
        # Chunk = 3 seconds video, 1 second = Frame Rate, Frames: seconds x Frame rate
        max_frames_per_chunk = 95  # This value can be adjusted as needed according to 15 FPS or 30FPS
        print(f"Processing {dataset_sub_dir} dataset...")
        video_frame_extractor.process_annotations(dataset_sub_dir, max_frames_per_chunk)

    end_time = time.time()
    time_taken = format_time(end_time - start_time)
    print(f"Time taken for extracting dataset frames: {time_taken}")

if __name__ == "__main__":
    main()


Processing train dataset...
Processing test dataset...
Processing val dataset...
Time taken for extracting dataset frames: 1 hours, 27 minutes, 57 seconds
